In [ ]:
#!pip install openai

     |████████████████████████████████| 41 kB 181 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 162 kB 22.7 MB/s 
  Created wheel for openai: filename=openai-0.16.0-py3-none-any.whl size=50784 sha256=16525f3f4b98be3f1448078f9045e8cd3693ac2e424534fa2327e1b6fbe4b74e
  Stored in directory: /root/.cache/pip/wheels/c1/f3/50/adfd6d5b5a417fef651921a8c5f77c0e644265ae000f3fb69b
Successfully built openai


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import openai
import pandas as pd
import configparser
from openai.api_resources import engine

In [2]:
config=configparser.ConfigParser()
config.read('openai_api_key.ini')
API_KEY=config['DEFAULT']['openai_api_key']

In [3]:
df=pd.read_pickle('Jesus_sayings_final_v6.pkl')

In [18]:
#show the first 10 lines of the dataframe
df.head(10)

,Version,Book,Chapter,Verse,Text
0,AKJV,Matthew,3,15,"And Jesus answering said unto him, Suffer it t..."
1,AKJV,Matthew,4,4,"But he answered and said, It is written, Man s..."
2,AKJV,Matthew,4,7,"Jesus said unto him, It is written again, Thou..."
3,AKJV,Matthew,4,10,"Then saith Jesus unto him, Get thee hence, Sat..."
4,AKJV,Matthew,4,17,"From that time Jesus began to preach, and to s..."
5,AKJV,Matthew,4,19,"And he saith unto them, Follow me, and I will ..."
6,AKJV,Matthew,5,3,Blessed are the poor in spirit: for theirs is ...
7,AKJV,Matthew,5,4,Blessed are they that mourn: for they shall be...
8,AKJV,Matthew,5,5,Blessed are the meek: for they shall inherit t...
9,AKJV,Matthew,5,6,Blessed are they which do hunger and thirst af...


In [6]:
def send_openai(quote):
  openai.api_key=API_KEY
  response = openai.Completion.create(
    model="ada:ft-appalachian-state-university-rel-st-2022-01-31-21-03-21",
    prompt="Extract the quote from the following passage:\n"+quote+"\nQuote:",
    temperature=0.7,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["END"]
  )
  # trim excess spaces from response['choices'][0]['text']
  return response['choices'][0]['text'].strip()


In [7]:
row=df.iloc[2]
print(row['Text'])
response=send_openai(row['Text'])
print(response)

Jesus said unto him, It is written again, Thou shalt not tempt the Lord thy God.
It is written again, Thou shalt not tempt the Lord thy God.


In [27]:
r=response["choices"][0]['text']
r

' And Jesus answering said unto him, Suffer it to be so now: for thus it becometh us to fulfil all righteousness. '

In [19]:
import sys, time
out=[]
stop=False
for idx,row in df.iterrows():
  print(row['Text'])
  quote=send_openai(row['Text'])
  print(quote)
  print(f"len of text={len(row['Text'].split(' '))} len of quote={len(quote.split(' '))}")
  if len(quote.split(' ')) in range(len(row['Text'].split(' '))-2,len(row['Text'].split(' '))+2):
    qerror=True
  else:
    qerror=False
  print(f'error={qerror}')
  if stop==False:
    #pause for 1 second
    print('pausing for 1 second')
    sys.stdout.flush()
    time.sleep(1)
    x=input("Press c to continue or x to exit")
    if x=='x':
      stop=True
      break
    if x=="c":
      stop=True
    if x=="x":
      sys.exit(0)
  out.append([{'Version':row['Version'],'Book':row['Book'],'Chapter':row['Chapter'],'Verse':row['Verse'],'Text':row['Text'],'Quote':quote,'Error':qerror}])
  print("---------------\n")
  quote=""

And Jesus answering said unto him, Suffer it to be so now: for thus it becometh us to fulfil all righteousness. Then he suffered him. 
And Jesus answering said unto him, Suffer it to be so now: for thus it becometh us to fulfil all righteousness.
len of text=26 len of quote=21
error=False
pausing for 1 second
---------------

But he answered and said, It is written, Man shall not live by bread alone, but by every word that proceedeth out of the mouth of God.
It is written, Man shall not live by bread alone, but by every word that proceedeth out of the mouth of God.
len of text=27 len of quote=22
error=False
pausing for 1 second
---------------

Jesus said unto him, It is written again, Thou shalt not tempt the Lord thy God.
It is written again, Thou shalt not tempt the Lord thy God.
len of text=16 len of quote=12
error=False
pausing for 1 second
---------------

Then saith Jesus unto him, Get thee hence, Satan: for it is written, Thou shalt worship the Lord thy God, and him only shalt 

In [15]:
print(out)

[[{'Version': 'AKJV', 'Book': 'Matthew', 'Chapter': '3', 'Verse': '15', 'Text': 'And Jesus answering said unto him, Suffer it to be so now: for thus it becometh us to fulfil all righteousness. Then he suffered him. ', 'Quote': 'And Jesus answering said unto him, Suffer it to be so now: for thus it becometh us to fulfil all righteousness.', 'Error': False}], [{'Version': 'AKJV', 'Book': 'Matthew', 'Chapter': '4', 'Verse': '4', 'Text': 'But he answered and said, It is written, Man shall not live by bread alone, but by every word that proceedeth out of the mouth of God.', 'Quote': 'But he answered and said, It is written, Man shall not live by bread alone, but by every word that proceedeth out of the mouth of God.', 'Error': True}], [{'Version': 'AKJV', 'Book': 'Matthew', 'Chapter': '4', 'Verse': '7', 'Text': 'Jesus said unto him, It is written again, Thou shalt not tempt the Lord thy God.', 'Quote': 'It is written again, Thou shalt not tempt the Lord thy God.', 'Error': False}], [{'Versi